In [27]:
import pandas as pd 
from ipywidgets import IntProgress


In [24]:
dkDF = pd.read_csv('data/DK_Predictions_Raw.csv', header = None, names = ['ContestId', 'MinutesRemaining', 'PacerEstimate'])#.drop(columns = ['SnapshotDate'])
contests = pd.read_csv('data/WorkingData.csv', encoding='cp1252').set_index('ContestId')[['MaxNumberPlayers', 'EntryFeeAmount']]
successDF = pd.read_csv('data/WorkedData.csv').set_index('ContestId')[['Success']]

In [37]:
contestBar = IntProgress(min=0, max=len(dkDF['ContestId'].unique()))
print("Contests:")
display(contestBar)

results = []
for cid in dkDF['ContestId'].unique():
    seriesDF = dkDF[dkDF['ContestId']==cid]
    seriesDF = seriesDF[seriesDF['MinutesRemaining']>240]
    if(len(seriesDF)>0):
        results.append({'ContestId':cid, 'PacerEstimate':seriesDF.iloc[0].PacerEstimate})
    contestBar.value+=1

Contests:


IntProgress(value=0, max=165614)

In [42]:
dkDF = pd.DataFrame(results).set_index('ContestId')

In [41]:
contests['SuccessThreshold'] = contests['EntryFeeAmount']*contests['MaxNumberPlayers']

In [43]:
mergedDF = pd.merge(contests, dkDF, on='ContestId', how='left').dropna()
mergedDF = pd.merge(mergedDF, successDF, on='ContestId', how='left').dropna()

In [44]:
mergedDF['PacerSuccess'] = (mergedDF['PacerEstimate'] + mergedDF['SuccessThreshold'])/mergedDF['SuccessThreshold']

In [46]:
mergedDF[['Success', 'PacerSuccess']].to_csv('data/DK_Pacer_Success.csv')